In [3]:
# SBERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지 sentence_transformers 설치
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 36.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4a92eb1ed8c167b14c568ca3929162513937386f71e634a930ee47b45c069fc6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [4]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [6]:
# 문장 임베딩을 얻기 위해 사전 훈련된 BERT 로드
# 여기선 한국어 포함되어 학습된 다국어 모델 로드

# 모델 이름 : 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
# 이름이 의미하는 바는 100가지 언어(한국어포함)지원하는 다국어 BERT BASE 모델
# SNLI데이터 학습 후 STS-B 데이터로 학습
# 문장 표현을 얻기 위해 평균풀링(mean-tokens) 사용
# -> NLI 데이터를 학습 후 STS데이터로 추가 파인튜닝한 모델

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [7]:
# 데이터의 모든 질문열 즉 train_data['Q']에 대해 문장 임베딩 값 구한 후 embedding이라는 새로운 열에 저장

train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [8]:
# 두 개의 벡터로부터 코사인 유사도 구하는 함수 정의

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [10]:
# 임의의 질문이 들어오면 
# 해당 질문의 문장 임베딩 값과 
# 챗봇 데이터의 임베딩 열, 즉 train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을
# 전부 비교 후 코사인 유사도 값이 가장 높은 질문 샘플 추출

# 해당 질문 샘플과 짝이 되는 답변 샘플 리털

def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

In [ ]:
# 챗봇 테스트

In [11]:
return_answer('나랑 밥먹자')

'맛있게 드세요.'

In [12]:
return_answer('나 오늘 심심해')

'답답한가봅니다.'

In [13]:
return_answer('나랑 같이 공부하자')

'공부는 언제나 좋죠.'